In [252]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from aif360.algorithms.inprocessing import GerryFairClassifier
from sklearn.svm import SVC
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import sklearn

import aif360
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult, get_distortion_german, get_distortion_compas

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas

from aif360.datasets import AdultDataset, BankDataset, CompasDataset, GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms import preprocessing, inprocessing, postprocessing
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.lfr import LFR
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
import copy

from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
%load_ext jupyternotify
np.random.seed(1)

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [136]:
def run_classification_metrics(CM:ClassificationMetric):
    def f1_score(priv=None):
            numer = CM.num_true_positives(privileged=priv)
            denom = CM.num_true_positives(privileged=priv) + 0.5*float(CM.num_false_positives(privileged=priv) + CM.num_false_negatives(privileged=priv))
            return float(numer/denom)
    return np.array([
        round(CM.accuracy(), 4),
        round(CM.theil_index(), 4),
        round(CM.false_positive_rate(privileged=False), 4),
        round(CM.false_positive_rate(privileged=True), 4),
        round(CM.false_negative_rate(privileged=False), 4),
        round(CM.false_negative_rate(privileged=True), 4),
        round(1-CM.error_rate(privileged=False), 4),
        round(1-CM.error_rate(privileged=True), 4),
        round(CM.false_discovery_rate(privileged=False), 4),
        round(CM.false_discovery_rate(privileged=True), 4),
        round(CM.false_omission_rate(privileged=False), 4),
        round(CM.false_omission_rate(privileged=True), 4),
        
        #all results
        CM.num_true_positives(),
        CM.num_true_negatives(),
        CM.num_false_positives(),
        CM.num_false_negatives(),
        
        #privileged
        CM.num_true_positives(privileged=True),
        CM.num_true_negatives(privileged=True),
        CM.num_false_positives(privileged=True),
        CM.num_false_negatives(privileged=True),
        
        #unprivileged
        CM.num_true_positives(privileged=False),
        CM.num_true_negatives(privileged=False),
        CM.num_false_positives(privileged=False),
        CM.num_false_negatives(privileged=False),
        
        round(f1_score(), 4),
        round(f1_score(True), 4),
        round(f1_score(False), 4),
    ])

In [245]:
def run_binary_dataset_metrics(BLDM:BinaryLabelDatasetMetric):
    #print("Consistency: ", BLDM.consistency())
    return np.array([
        round(BLDM.base_rate(privileged=True), 4), # 1 means privileged bias
        round(BLDM.base_rate(privileged=False), 4), # 1 means unprivileged bias
        round(BLDM.consistency()[0], 4)
    ])

In [180]:
def get_model_name(model):
    if isinstance(model, sklearn.linear_model.LogisticRegression):
        return "Logistic Regression"
    if isinstance(model, sklearn.linear_model.LinearRegression):
        return "Linear Regression"
    if isinstance(model, sklearn.ensemble.BaggingClassifier):
        return "Meta Classifier"
    
    if isinstance(model, preprocessing.DisparateImpactRemover):
        return "DIR"
    if isinstance(model, preprocessing.LFR):
        return "LFR"
    if isinstance(model, preprocessing.OptimPreproc):
        return "OP"
    if isinstance(model, preprocessing.Reweighing):
        return "RW"
    
    if isinstance(model, inprocessing.PrejudiceRemover):
        return "PR"
    if isinstance(model, inprocessing.AdversarialDebiasing):
        return "AD"
    if isinstance(model, inprocessing.ARTClassifier):
        return "ARTC"
    if isinstance(model, inprocessing.ExponentiatedGradientReduction):
        return "EGR"
    if isinstance(model, inprocessing.GerryFairClassifier):
        return "GFC"
    if isinstance(model, inprocessing.GridSearchReduction):
        return "GSR"
    if isinstance(model, inprocessing.MetaFairClassifier):
        return "MFC"
    
    if isinstance(model, postprocessing.EqOddsPostprocessing):
        return "EOP"
    if isinstance(model, postprocessing.CalibratedEqOddsPostprocessing):
        return "CEOP"
    if isinstance(model, postprocessing.RejectOptionClassification):
        return "ROC"
    
    return "None"

In [181]:
def get_dataset_name(dataset):
    if isinstance(dataset, aif360.datasets.german_dataset.GermanDataset):
        return "German Dataset"
    if isinstance(dataset, aif360.datasets.adult_dataset.AdultDataset):
        return "Adult Dataset"
    if isinstance(dataset, aif360.datasets.bank_dataset.BankDataset):
        return "Bank Dataset"
    if isinstance(dataset, aif360.datasets.compas_dataset.CompasDataset):
        return "Compas Dataset"

In [262]:
def analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, classifier=None, 
                 preprocessing_algo=None, inprocessing_algo=None, postprocessing_algo=None):
    print(get_model_name(preprocessing_algo), get_model_name(inprocessing_algo), get_model_name(postprocessing_algo))

    model = sklearn.linear_model.LogisticRegression() # solver='liblinear', class_weight='balanced', 
    
    dataset_train_pred = dataset_train.copy(deepcopy=True)
    dataset_test_pred = dataset_test.copy(deepcopy=True)
    
    '''
    scale_orig = StandardScaler()
    dataset_train.features = scale_orig.fit_transform(dataset_train.features)
    dataset_test.features = scale_orig.fit_transform(dataset_test.features)
    dataset_train_pred.features = scale_orig.fit_transform(dataset_train_pred.features)
    dataset_test_pred.features = scale_orig.fit_transform(dataset_test_pred.features)
    '''   

    #if preprocessing_algo is None and inprocessing_algo is None and postprocessing_algo is None:
    X_train = dataset_train_pred.features
    y_train = dataset_train_pred.labels.ravel()

    model = model.fit(X_train, y_train)

    fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
    # scores for train dataset required for post-processing algos
    dataset_train_pred.scores = model.predict_proba(dataset_train_pred.features)[:,fav_idx].reshape(-1,1) 
    dataset_train_pred.labels = model.predict(dataset_train_pred.features).reshape(-1,1) 
    dataset_test_pred.scores = model.predict_proba(dataset_test_pred.features)[:,fav_idx].reshape(-1,1) 
    dataset_test_pred.labels = model.predict(dataset_test_pred.features).reshape(-1,1) 
    
    #class_thresh = 0.5
    #y_test_pred = np.zeros_like(dataset_test_pred.labels)
    #y_test_pred[dataset_test_pred.scores >= class_thresh] = dataset_test_pred.favorable_label
    #y_test_pred[~(dataset_test_pred.scores >= class_thresh)] = dataset_test_pred.unfavorable_label
    #dataset_test_pred.labels = y_test_pred
        
    if preprocessing_algo is not None:
        print("Pre-processing")
        #dataset_train_pred = preprocessing_algo.fit_transform(dataset_train_pred)
        #dataset_train_pred = dataset_test_pred.align_datasets(dataset_train_pred)
        
        dataset_train_pred = preprocessing_algo.fit_transform(dataset_train_pred)
        dataset_train_pred = dataset_train.align_datasets(dataset_train_pred)
        
        #dataset_test_pred = preprocessing_algo.fit_transform(dataset_test_pred)
        
        model.fit(dataset_train_pred.features, dataset_train_pred.labels.ravel())
        # sample_weight=dataset_train_pred.instance_weights

        fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
        dataset_train_pred.scores = model.predict_proba(dataset_train_pred.features)[:,fav_idx].reshape(-1,1) 
        dataset_train_pred.labels = model.predict(dataset_train_pred.features).reshape(-1,1) 
        dataset_test_pred.scores = model.predict_proba(dataset_test_pred.features)[:,fav_idx].reshape(-1,1) 

        #class_thresh = 0.5
        #y_test_pred = np.zeros_like(dataset_test_pred.labels)
        #y_test_pred[dataset_test_pred.scores >= class_thresh] = dataset_test_pred.favorable_label
        #y_test_pred[~(dataset_test_pred.scores >= class_thresh)] = dataset_test_pred.unfavorable_label
        #dataset_test_pred.labels = y_test_pred
        dataset_test_pred.labels = model.predict(dataset_test_pred.features).reshape(-1,1)  
    
    if inprocessing_algo is not None:
        model = inprocessing_algo
        model.fit(dataset_train_pred)
        dataset_train_pred = model.predict(dataset_train_pred)
        dataset_test_pred = model.predict(dataset_test_pred) 
        
    if postprocessing_algo is not None:
        pp = postprocessing_algo
        pp = pp.fit(dataset_train, dataset_train_pred)
        dataset_test_pred = pp.predict(dataset_test_pred)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    
    BLDM = BinaryLabelDatasetMetric(dataset_test_pred,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    name = ""
    if preprocessing_algo is not None:
        name += get_model_name(preprocessing_algo) + " + "
    if inprocessing_algo is not None:
        name += get_model_name(inprocessing_algo) + " + "
    if postprocessing_algo is not None:
        name += get_model_name(postprocessing_algo)
    if name == "":
        name = get_model_name(model)
        
    if name.endswith(" + "):
        lastIndex = name.rindex(" + ")
        name = name[:lastIndex]
    
    #print(run_classification_metrics(CM))
    #print(run_binary_dataset_metrics(BLDM))
    metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
       
    return {"key":name, "val":metrics}

In [263]:
def get_dataset_options(dataset_name):
    optim_options = {
        "epsilon": 0.05,
        "clist": [0.99, 1.99, 2.99],
        "dlist": [.1, 0.05, 0]
    }
    if dataset_name=="adult":
        optim_options["distortion_fun"] = get_distortion_adult
        pro_attr = 'sex'
        return (load_preproc_data_adult(['sex']), pro_attr, [{'sex': 1}], [{'sex': 0}], optim_options)
    elif dataset_name=="compas":
        optim_options["distortion_fun"] = get_distortion_compas
        pro_attr = 'race'
        return (load_preproc_data_compas(['race']), pro_attr, [{'race': 1}], [{'race': 0}], optim_options)
    elif dataset_name=="bank":
        pro_attr = 'age'
        return (BankDataset(protected_attribute_names=['age'],
            privileged_classes=[lambda x: x >= 25], 
            features_to_drop=['day_of_week']), pro_attr, [{'age': 1}], [{'age': 0}], None)
    elif dataset_name=="german":
        pro_attr = 'age'
        optim_options["distortion_fun"] = get_distortion_german
        return (load_preproc_data_german(['age']), pro_attr, 
                    [{'age': 1}], [{'age': 0}], optim_options)

In [264]:
#datasets = ["adult", "compas", "german", "bank"]
datasets = ["adult", "compas", "bank"]
for dataset_name in datasets:
    dataset, pro_attr, privileged_groups, unprivileged_groups, optim_options = get_dataset_options(dataset_name)
    
    print("DATASET NAME: ", dataset_name)
    np.random.seed(123)
    dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

    #preprocessing_algos = [LFR(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, k=10, Ax=0.1, Ay=1.0, Az=2.0, verbose=0),
    #                       OptimPreproc(OptTools, optim_options, unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups),
                          #preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
    #                      preprocessing.DisparateImpactRemover(),
    #                      None]
                          
    preprocessing_algos = [None]

    inprocessing_algos = [GerryFairClassifier(),
                          inprocessing.PrejudiceRemover(sensitive_attr=pro_attr),
                          inprocessing.ExponentiatedGradientReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                          inprocessing.GridSearchReduction(sklearn.linear_model.LogisticRegression(), prot_attr=pro_attr, constraints="DemographicParity", drop_prot_attr=False),
                          ##inprocessing.MetaFairClassifier(),
                          None]
    #inprocessing_algos = [None]
    #postprocessing_algos = [postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0),
    #                        postprocessing.CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups, seed=0),
    #                        postprocessing.RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),  
    #                        None]

    postprocessing_algos = [None]

    df = {}

    for pre in preprocessing_algos:
        for inproc in inprocessing_algos:
            for post in postprocessing_algos:
                try:
                    res = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, 
                             preprocessing_algo=copy.deepcopy(pre), 
                             inprocessing_algo=copy.deepcopy(inproc),
                             postprocessing_algo=copy.deepcopy(post))
                    df[res["key"]] = res["val"]

                except KeyboardInterrupt:
                    raise KeyboardInterrupt()
                #except Exception as e:
                #    print("FAILED: " + get_model_name(pre) + ", " + get_model_name(inproc) + ", " + get_model_name(post) + " on dataset " + get_dataset_name(dataset), e)

    df = pd.DataFrame.from_dict(df)                
    df.index = ["Accuracy", "Theil Index",
                    "False Positive Rate - Unprivileged", "False Positive Rate - Privileged",
                    "False Negative Rate - Unprivileged", "False Negative Rate - Privileged",
                    "Accuracy - Unprivileged", "Accuracy - Privileged",
                    "False Discovery Rate - Unprivileged", "False Discovery Rate - Privileged",
                    "False Omission Rate - Unprivileged", "False Omission Rate - Privileged",
                    "Num True Pos", "Num True Neg", "Num False Pos", "Num False Neg",
                    "Num True Pos - Privileged", "Num True Neg - Privileged", "Num False Pos - Privileged", "Num False Neg - Privileged",
                    "Num True Pos - Unprivileged", "Num True Neg - Unprivileged", "Num False Pos - Unprivileged", "Num False Neg - Unprivileged",
                    "F1 Score", "F1 Score - Privileged", "F1 Score - Unprivileged",
                    "Privileged base Rate", "Unprivileged base Rate", "Consistency"]

    df = df.T
    df.to_csv("Data -"+ dataset_name +".csv", sep=',', encoding='utf-8')
    display(df)        

DATASET NAME:  adult
None GFC None
In-processing
None PR None
In-processing


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [277]:
#g = load_preproc_data_compas(['race'])  
g = load_preproc_data_german(['age'])
#g = load_preproc_data_adult(['sex'])

np.random.seed(123)
dataset_train, dataset_test = g.split([0.7], shuffle = True)
#model = sklearn.linear_model.LogisticRegression() # solver='liblinear', class_weight='balanced', 
#dataset_train.scores.ravel() #== dataset_train.labels.ravel()
#model = GerryFairClassifier()
model = inprocessing.PrejudiceRemover(sensitive_attr='age',  class_attr='credit')

model.fit(dataset_train)
#tr = model.predict(dataset_train)
model.predict(dataset_test)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed